# Intertidal Bathymetry
Tests intertidal bathymetry function of `bathymetry.py`. 

In [1]:
from pathlib import Path
import sys
import time

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry

In [33]:
Map = geemap.Map(center=(38.499573, -8.89696), zoom=13)
bounds = ee.Geometry(Polygon([[
    (-8.9793574609375, 38.474681402665055),
    (-8.8145625390625, 38.474681402665055),
    (-8.8145625390625, 38.5244559986627),
    (-8.9793574609375, 38.5244559986627),
    (-8.9793574609375, 38.474681402665055),
]]))

In [34]:
sdb = bathymetry.Bathymetry()

scale = Map.getScale()

# Parameters 
start_date = '2018-01-01'
stop_date = '2020-01-01'

image = sdb.compute_intertidal_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale,
    missions=['S2', 'L8'],
    # filter: ee.Filter.dayOfYear(7*30, 9*30), # summer-only
    filter_masked=False, 
    # filterMasked: true, 
    # filterMaskedFraction: 0.5,
    skip_scene_boundary_fix=True,
    skip_neighborhood_search=True,
    neighborhood_search_parameters={"erosion": 0, "dilation": 0, "weight": 50},
    bounds_buffer=0,
    water_index_min=-0.05,
    water_index_max=0.15,

    # lowerCdfBoundary: 45,
    # upperCdfBoundary: 50
)

min = 0.04
max = 0.2
Map.addLayer(sdb.composite, { "min": min, "max": max }, 'red-green-blue')
Map

Map(center=[38.499573, -8.89696], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [35]:
Map.addLayer(image, {"min": sdb.waterIndexMin, "max": sdb.waterIndexMax}, 'water indices', False)